# Azure AI Agents with Model Context Protocol (MCP) Support

यो नोटबुकले Azure AI Agents लाई Model Context Protocol (MCP) उपकरणहरूसँग प्रयोग गरेर एक बुद्धिमान एजेन्ट सिर्जना गर्ने तरिका देखाउँछ, जसले बाह्य MCP सर्भरहरूको प्रयोग गरेर थप क्षमताहरू प्राप्त गर्न सक्छ।


## आवश्यक NuGet प्याकेजहरू स्थापना गर्नुहोस्

पहिले, हामीले Azure AI Agents Persistent प्याकेज स्थापना गर्नुपर्छ, जसले Azure AI Agents सँग काम गर्नको लागि मुख्य कार्यक्षमता प्रदान गर्दछ।


## कीलेस प्रमाणीकरणका फाइदाहरू

यो नोटबुकले **कीलेस प्रमाणीकरण** प्रदर्शन गर्दछ, जसले धेरै फाइदाहरू प्रदान गर्दछ:
- ✅ **API कीहरू व्यवस्थापन गर्न आवश्यक छैन** - Azure पहिचान-आधारित प्रमाणीकरण प्रयोग गर्दछ
- ✅ **सुरक्षा वृद्धि** - कोड वा कन्फिगरेसनमा कुनै गोप्य जानकारी भण्डारण हुँदैन
- ✅ **स्वचालित प्रमाणपत्र परिक्रमा** - Azure ले प्रमाणपत्र जीवनचक्र व्यवस्थापन गर्दछ
- ✅ **भूमिका-आधारित पहुँच** - सूक्ष्म अनुमति व्यवस्थापनका लागि Azure RBAC प्रयोग गर्दछ

`DefaultAzureCredential` ले स्वचालित रूपमा उपलब्ध सबैभन्दा राम्रो प्रमाणपत्र स्रोत प्रयोग गर्दछ:
1. प्रबन्धित पहिचान (Azure मा चल्दा)
2. Azure CLI प्रमाणपत्रहरू (विकासको क्रममा)
3. Visual Studio प्रमाणपत्रहरू
4. वातावरण चरहरू (यदि कन्फिगर गरिएको छ)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## आवश्यक नामस्थानहरू आयात गर्नुहोस्

Azure AI Agents र Azure Identity का लागि आवश्यक नामस्थानहरू आयात गर्नुहोस्।


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI एजेन्ट क्लाइन्ट (किलेस प्रमाणीकरण) कन्फिगर गर्नुहोस्

कन्फिगरेसन भेरिएबलहरू सेटअप गर्नुहोस् र **किलेस प्रमाणीकरण** प्रयोग गरेर PersistentAgentsClient सिर्जना गर्नुहोस्:
- **projectEndpoint**: Azure AI Foundry परियोजना अन्त बिन्दु
- **modelDeploymentName**: परिनियोजित AI मोडेलको नाम (GPT-4.1 नानो)
- **mcpServerUrl**: MCP सर्भरको URL (Microsoft Learn API)
- **mcpServerLabel**: MCP सर्भरलाई चिन्हित गर्न प्रयोग गरिने लेबल
- **DefaultAzureCredential**: व्यवस्थापित पहिचान, Azure CLI, वा अन्य प्रमाणिकरण स्रोतहरू प्रयोग गर्दछ (API कुञ्जीहरू आवश्यक छैन)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP उपकरण परिभाषा बनाउनुहोस्

Microsoft Learn MCP सर्भरमा जडान हुने MCP उपकरण परिभाषा बनाउनुहोस्। यसले एजेन्टलाई Microsoft Learn सामग्री र दस्तावेजहरू पहुँच गर्न अनुमति दिनेछ।


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## एआई एजेन्ट बनाउनुहोस्

निर्दिष्ट मोडेल र MCP उपकरणहरू प्रयोग गरेर स्थायी एआई एजेन्ट बनाउनुहोस्। एजेन्ट निम्न प्रकारले कन्फिगर गरिएको छ:
- GPT-4.1 नानो मोडेल
- सहायता प्रदान गर्न MCP उपकरणहरू प्रयोग गर्ने निर्देशनहरू
- Microsoft Learn MCP सर्भरमा पहुँच


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## थ्रेड सिर्जना गर्नुहोस् र सन्देश पठाउनुहोस्

कुराकानी थ्रेड सिर्जना गर्नुहोस् र प्रयोगकर्तालाई Azure OpenAI र OpenAI बीचको फरकबारे सोध्ने सन्देश पठाउनुहोस्। यसले MCP उपकरणहरूको प्रयोग गरेर सही जानकारी प्रदान गर्ने एजेन्टको क्षमता परीक्षण गर्नेछ।


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP उपकरण स्रोतहरू कन्फिगर गर्नुहोस् (कुञ्जीबिहीन)

MCP उपकरण स्रोतहरू सेटअप गर्नुहोस्। पूर्ण रूपमा कुञ्जीबिहीन दृष्टिकोणका लागि, यदि MCP सर्भरले Azure पहिचान-आधारित प्रमाणीकरणलाई समर्थन गर्दछ भने, तपाईं अनुकूलित हेडरहरू हटाउन सक्नुहुन्छ। तलको उदाहरणले आवश्यक परेमा हेडरहरू कसरी थप्ने देखाउँछ, तर कुञ्जीबिहीन परिदृश्यहरूका लागि, यी आवश्यक नहुन सक्छन्।


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## एजेन्ट रन सुरु गर्नुहोस्

प्रयोगकर्ताको सन्देश प्रक्रिया गर्न रन सिर्जना र सुरु गर्नुहोस्। एजेन्टले प्रतिक्रिया उत्पन्न गर्न कन्फिगर गरिएको MCP उपकरणहरू र स्रोतहरू प्रयोग गर्नेछ।


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## रन अनुगमन गर्नुहोस् र उपकरण स्वीकृतिहरू व्यवस्थापन गर्नुहोस् (किलेस)

एजेन्टको रन स्थिति अनुगमन गर्नुहोस् र आवश्यक उपकरण स्वीकृतिहरू व्यवस्थापन गर्नुहोस्। यो लूप:
1. रन पूरा हुने वा कुनै कार्य आवश्यक पर्ने प्रतीक्षा गर्दछ
2. आवश्यक परेमा MCP उपकरण कलहरू स्वचालित रूपमा स्वीकृत गर्दछ
3. किलेस प्रमाणीकरणको लागि, MCP सर्भरले Azure पहिचान समर्थन गरेमा हेडरहरू आवश्यक नहुन सक्छ


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## संवाद परिणाम प्रदर्शन गर्नुहोस्

थ्रेडमा भएका सबै सन्देशहरू प्राप्त गरेर प्रदर्शन गर्नुहोस्, जसमा प्रयोगकर्ताको प्रश्न र एजेन्टको उत्तर दुवै समावेश छन्। सन्देशहरू समयक्रम अनुसार टाइमस्ट्याम्प र भूमिका सूचकसहित देखाइन्छ।


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
